In [ ]:
#Libraries
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import cv2
import pandas as pd
from tqdm import tqdm
from random import shuffle
import keras 
from keras.models import Sequential, Model
from keras.layers import Conv2D, Dense, Activation, Dropout, Flatten, AveragePooling2D, BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
import seaborn as sns
from keras.utils import np_utils
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras import regularizers, initializers

In [ ]:
#Parameter to resize the image
IMG_SIZE=32

In [ ]:
# Creating a data file

LABEL_LIST = pd.read_csv('/home/aleksandra/Desktop/ML project/list_patition_label.txt', 
                         names=('Iname','Class'), sep=" ")

# Separate two datasets label
TRAIN_LIST = LABEL_LIST[LABEL_LIST['Iname'].str.contains("train")]
TEST_LIST = LABEL_LIST[LABEL_LIST['Iname'].str.contains("test")]
TEST_LIST = TEST_LIST.reset_index()
TEST_LIST = TEST_LIST.drop(columns='index')

# The path to the folder with images
folder_img = '/home/aleksandra/Desktop/ML project/aligned'

# Data set creation function
def create_data_set(path_file,list_label):
    data_set =[]
    for img in tqdm(os.listdir(path_file)):
        path = os.path.join(path_file,img)
        for i in range(len(list_label.index)):
            if img.split('_aligned.jpg')[0] == list_label.loc[i,'Iname'].split('.jpg')[0]:
                limg = cv2.imread(path, cv2.IMREAD_GRAYSCALE)   
                limg = cv2.resize(limg, (IMG_SIZE,IMG_SIZE))
                data_set.append([np.array(limg),list_label.loc[i,'Class']])
    return data_set

# Calling a function and saving the results for the training dataset
train_data = create_data_set(folder_img, TRAIN_LIST) 
shuffle(train_data)
np.save('/home/aleksandra/Desktop/ML project/train_data.npy', train_data)

# Calling a function and saving the results for the test dataset
test_data = create_data_set(folder_img, TEST_LIST)
shuffle(test_data)
np.save('/home/aleksandra/Desktop/ML project/test_data.npy', test_data)


In [ ]:
#Upload dataset
train_data = np.load('/home/aleksandra/Desktop/ML project/train_data.npy',allow_pickle=True)
test_data = np.load('/home/aleksandra/Desktop/ML project/test_data.npy', allow_pickle=True)

In [ ]:
#Splitting data for data training and validation
train = train_data[:-2500]
test = train_data[-2500:]

In [ ]:
#Resize and normalize data training and data validation
train_x = np.array([i[0]/255 for i in train]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
train_y = np.array([i[1]-1 for i in train])
train_y = np_utils.to_categorical(train_y)

test_x = np.array([i[0]/255 for i in test]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
test_y = np.array([i[1]-1 for i in test])
test_y = np_utils.to_categorical(test_y)

In [ ]:
# Classical Lenet-5 model 
def Lenet5_classic(train_x, train_y,test_x,test_y):
    model1 = keras.Sequential()
    model1.add(Conv2D(filters=6, kernel_size=(3, 3), activation='relu', input_shape=(32,32,1)))
    model1.add(AveragePooling2D(strides=2))
    model1.add(Conv2D(filters=16, kernel_size=(3, 3), activation='relu'))
    model1.add(AveragePooling2D(strides=2))
    model1.add(Flatten())
    model1.add(Dense(units=120, activation='relu'))
    model1.add(Dense(units=84, activation='relu'))
    model1.add(Dense(units=7, activation = 'softmax'))
    model1.build()
    model1.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
    history = model1.fit(train_x, train_y, batch_size=128, epochs=10,validation_data=(test_x,test_y),callbacks=[EarlyStopping(monitor='val_loss', patience=5, verbose=1)])
    model1.save("lenet5_classic.h5")
    np.save('history_classic.npy',history.history)
    return(model1,history)

# Call a function
model_c, history_c = Lenet5_classic(train_x, train_y, test_x, test_y)

# Normalize test data
test_X = np.array([i[0]/255 for i in test_data]).reshape(-1,IMG_SIZE,IMG_SIZE,1)

# Evaluate model
prediction = model_c.predict(test_X)
y_pred = np.argmax(prediction, axis=1)
y_test = np.array([i[1]-1 for i in test_data])
print('Test set error rate: {}'.format(np.mean(y_pred == y_test)))
num_classes=7
target_names = ["Class {}".format(i) for i in range(num_classes)]
print(classification_report(y_test, y_pred, target_names=target_names))


In [ ]:
# Lenet-5 model with dropout
def Lenet5_dropout(train_x, train_y,test_x,test_y):
    model1 = keras.Sequential()
    model1.add(Conv2D(filters=6, kernel_size=(3, 3), activation='relu', input_shape=(32,32,1)))
    model1.add(AveragePooling2D(strides=2))
    model1.add(Conv2D(filters=16, kernel_size=(3, 3), activation='relu'))
    model1.add(AveragePooling2D(strides=2))
    model1.add(Flatten())
    model1.add(Dense(units=120, activation='relu'))
    model1.add(Dropout(0.4))
    model1.add(Dense(units=84, activation='relu'))
    model1.add(Dropout(0.4))
    model1.add(Dense(units=7, activation = 'softmax'))
    model1.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
    history = model1.fit(train_x, train_y, batch_size=128, epochs=40,validation_data=(test_x,test_y),callbacks=[EarlyStopping(monitor='val_loss', patience=5, verbose=1)])
    model1.save("lenet5_dropout.h5")
    np.save('history_dropout.npy',history.history)
    return(model1,history)

# Call a function
model_d, history_d = Lenet5_dropout(train_x, train_y, test_x, test_y)

# Normalize test data
test_X = np.array([i[0]/255 for i in test_data]).reshape(-1,IMG_SIZE,IMG_SIZE,1)

# Evaluate model
prediction = model_d.predict(test_X)
y_pred = np.argmax(prediction, axis=1)
y_test = np.array([i[1]-1 for i in test_data])
print('Test set error rate: {}'.format(np.mean(y_pred == y_test)))
num_classes=7
target_names = ["Class {}".format(i) for i in range(num_classes)]
print(classification_report(y_test, y_pred, target_names=target_names))


In [ ]:
# Lenet-5 model with adding two additional convolutional layers
def Lenet5_model1(train_x, train_y,test_x,test_y):
    model1 = keras.Sequential()
    model1.add(Conv2D(filters=6, kernel_size=(3,3), input_shape=(32,32,1), activation='relu')
    model1.add(Conv2D(filters=12, kernel_size=(4,4), activation='relu')
    model1.add(AveragePooling2D(strides=2))
    model1.add(Conv2D(filters=16, kernel_size=(3,3), activation='relu')
    model1.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu')
    model1.add(AveragePooling2D(strides=2))
    model1.add(Flatten())
    model1.add(Dense(units=120, activation='relu'
    model1.add(Dropout(0.4))
    model1.add(Dense(units=84, activation='relu')
    model1.add(Dropout(0.4))
    model1.add(Dense(units=7, activation = 'softmax')
    model1.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
    history = model1.fit(train_x, train_y, batch_size=128, epochs=40,validation_data=(test_x,test_y),callbacks=[EarlyStopping(monitor='val_loss', patience=5, verbose=1)])
    model1.save("lenet5_model1.h5")
    np.save('history_model1.npy',history.history)
    return(model1,history)

# Call a function
model_1, history_1 = Lenet5_model1(train_x, train_y, test_x, test_y)

# Normalize test data
test_X = np.array([i[0]/255 for i in test_data]).reshape(-1,IMG_SIZE,IMG_SIZE,1)

# Evaluate model
prediction = model_1.predict(test_X)
y_pred = np.argmax(prediction, axis=1)
y_test = np.array([i[1]-1 for i in test_data])
print('Test set error rate: {}'.format(np.mean(y_pred == y_test)))
num_classes=7
target_names = ["Class {}".format(i) for i in range(num_classes)]
print(classification_report(y_test, y_pred, target_names=target_names))


In [ ]:
# Lenet-5 model with using 3x3 filters on all convolutional layers
def Lenet5_model2(train_x, train_y,test_x,test_y):
    model1 = keras.Sequential()
    model1.add(Conv2D(filters=6, kernel_size=(3,3), input_shape=(32,32,1), activation='relu')
    model1.add(Conv2D(filters=12, kernel_size=(3,3), activation='relu')
    model1.add(AveragePooling2D(strides=2))
    model1.add(Conv2D(filters=16, kernel_size=(3,3), activation='relu')
    model1.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu')
    model1.add(AveragePooling2D(strides=2))
    model1.add(Flatten())
    model1.add(Dense(units=120, activation='relu'
    model1.add(Dropout(0.4))
    model1.add(Dense(units=84, activation='relu')
    model1.add(Dropout(0.4))
    model1.add(Dense(units=7, activation = 'softmax')
    model1.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
    history = model1.fit(train_x, train_y, batch_size=128, epochs=40,validation_data=(test_x,test_y),callbacks=[EarlyStopping(monitor='val_loss', patience=5, verbose=1)])
    model1.save("lenet5_model2.h5")
    np.save('history_model2.npy',history.history)
    return(model1,history)
              
# Call a function
model_2, history_2 = Lenet5_model2(train_x, train_y, test_x, test_y)

# Normalize test data
test_X = np.array([i[0]/255 for i in test_data]).reshape(-1,IMG_SIZE,IMG_SIZE,1)

# Evaluate model
prediction = model_2.predict(test_X)
y_pred = np.argmax(prediction, axis=1)
y_test = np.array([i[1]-1 for i in test_data])
print('Test set error rate: {}'.format(np.mean(y_pred == y_test)))
num_classes=7
target_names = ["Class {}".format(i) for i in range(num_classes)]
print(classification_report(y_test, y_pred, target_names=target_names))               

In [ ]:
# Lenet-5 model with BatchNormalization
def Lenet5_model3(train_x, train_y,test_x,test_y):
    model1 = keras.Sequential()
    model1.add(BatchNormalization(axis=1))
    model1.add(Conv2D(filters=6, kernel_size=(3,3), input_shape=(32,32,1), activation='relu')
    model1.add(Conv2D(filters=12, kernel_size=(3,3), activation='relu')
    model1.add(AveragePooling2D(strides=2))
    model1.add(BatchNormalization(axis=1))
    model1.add(Conv2D(filters=16, kernel_size=(3,3), activation='relu')
    model1.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu')
    model1.add(AveragePooling2D(strides=2))
    model1.add(BatchNormalization(axis=1))           
    model1.add(Flatten())
    model1.add(Dense(units=120, activation='relu'
    model1.add(Dropout(0.4))
    model1.add(Dense(units=84, activation='relu')
    model1.add(BatchNormalization(axis=1))
    model1.add(Dropout(0.4))
    model1.add(Dense(units=7, activation = 'softmax')
    model1.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
    history = model1.fit(train_x, train_y, batch_size=128, epochs=40,validation_data=(test_x,test_y),callbacks=[EarlyStopping(monitor='val_loss', patience=5, verbose=1)])
    model1.save("lenet5_model3.h5")
    np.save('history_model3.npy',history.history)
    return(model1,history)
  
# Call a function
model_3, history_3 = Lenet5_model3(train_x, train_y, test_x, test_y)

# Normalize test data
test_X = np.array([i[0]/255 for i in test_data]).reshape(-1,IMG_SIZE,IMG_SIZE,1)

# Evaluate model
prediction = model_3.predict(test_X)
y_pred = np.argmax(prediction, axis=1)
y_test = np.array([i[1]-1 for i in test_data])
print('Test set error rate: {}'.format(np.mean(y_pred == y_test)))
num_classes=7
target_names = ["Class {}".format(i) for i in range(num_classes)]
print(classification_report(y_test, y_pred, target_names=target_names))       

In [ ]:
# Lenet-5 model with application of He initializers, Xavier initializer,l2-regularizer
def Lenet5_model4(train_x, train_y,test_x,test_y):
    model1 = keras.Sequential()
    model1.add(BatchNormalization(axis=1))
    model1.add(Conv2D(filters=6, kernel_size=(3,3), input_shape=(32,32,1), activation='relu',kernel_initializer=initializers.HeUniform(), kernel_regularizer=regularizers.L2(1e-4),bias_initializer=initializers.HeUniform(),bias_regularizer=regularizers.L2(1e-4)))
    model1.add(Conv2D(filters=12, kernel_size=(3,3), activation='relu',kernel_initializer=initializers.HeUniform(), kernel_regularizer=regularizers.L2(1e-4),bias_initializer=initializers.HeUniform(),bias_regularizer=regularizers.L2(1e-4)))
    model1.add(AveragePooling2D(strides=2))
    model1.add(BatchNormalization(axis=1))
    model1.add(Conv2D(filters=16, kernel_size=(3,3), activation='relu', kernel_initializer=initializers.HeUniform(),bias_initializer=initializers.HeUniform(),bias_regularizer=regularizers.L2(1e-4), kernel_regularizer=regularizers.L2(1e-4)))
    model1.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu', kernel_initializer=initializers.HeUniform(),bias_initializer=initializers.HeUniform(),bias_regularizer=regularizers.L2(1e-4), kernel_regularizer=regularizers.L2(1e-4)))
    model1.add(AveragePooling2D(strides=2))
    model1.add(BatchNormalization(axis=1))
    model1.add(Flatten())
    model1.add(Dense(units=120, activation='relu',bias_initializer=initializers.HeUniform(),bias_regularizer=regularizers.L2(1e-4), kernel_regularizer=regularizers.L2(1e-4), kernel_initializer=initializers.HeUniform()))
    model1.add(Dropout(0.4))
    model1.add(Dense(units=84, activation='relu',bias_initializer=initializers.HeUniform(),bias_regularizer=regularizers.L2(1e-4), kernel_regularizer=regularizers.L2(1e-4), kernel_initializer=initializers.HeUniform()))
    model1.add(BatchNormalization(axis=1))
    model1.add(Dropout(0.4))
    model1.add(Dense(units=7, activation = 'softmax', kernel_initializer=initializers.GlorotUniform(),kernel_regularizer=regularizers.L2(1e-4),bias_initializer=initializers.GlorotUniform(),bias_regularizer=regularizers.L2(1e-4)))
    model1.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
    history = model1.fit(train_x, train_y, batch_size=128, epochs=40,validation_data=(test_x,test_y),callbacks=[EarlyStopping(monitor='val_loss', patience=5, verbose=1)])
    model1.save("lenet5_model4.h5")
    np.save('history_model4.npy',history.history)
    return(model1,history)

# Call a function
model_4, history_4 = Lenet5_model4(train_x, train_y, test_x, test_y)

# Normalize test data
test_X = np.array([i[0]/255 for i in test_data]).reshape(-1,IMG_SIZE,IMG_SIZE,1)

# Evaluate model
prediction = model_4.predict(test_X)
y_pred = np.argmax(prediction, axis=1)
y_test = np.array([i[1]-1 for i in test_data])
print('Test set error rate: {}'.format(np.mean(y_pred == y_test)))
num_classes=7
target_names = ["Class {}".format(i) for i in range(num_classes)]
print(classification_report(y_test, y_pred, target_names=target_names))       

In [ ]:
# The ensemble of model3 and model4

# Upload model3 and model4
model1 = tf.keras.models.load_model('lenet5_model3.h5')
model2 = tf.keras.models.load_model('lenet5_model4.h5')
model_list = [model1,model2]

def ensemble_prediction(members, testX):
    yhats = [model.predict(testX) for model in members]
    summed = np.sum(yhats, axis=0)
    result = np.argmax(summed, axis=1)
    return result

# Normalize test data
test_X = np.array([i[0]/255 for i in test_data]).reshape(-1,IMG_SIZE,IMG_SIZE,1)

# Evaluate model
y_pred = ensemble_prediction(model_list, test_X)
y_test = np.array([i[1]-1 for i in test_data])
print('Test set error rate: {}'.format(np.mean(y_pred == y_test)))
num_classes=7
target_names = ["Class {}".format(i) for i in range(num_classes)]
print(classification_report(y_test, y_pred, target_names=target_names))
